In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.metrics.pairwise import cosine_similarity

from utility.progress_bar import update_progress_bar

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


In [14]:
import os
import pathlib
from utility.hashing import convert_to_number, convert_from_number

def save_vectors(sentences):
    
    max_length = len(sentences)
    counter = 0
    update_progress_bar(0, max_length, prefix='Progress:', suffix='Complete', length=50)
    
    
    pathlib.Path(os.path.join('ml_models', 'elmo', 'sick', '1')).mkdir(parents=True, exist_ok=True) 

    for sent in sentences:
        init = tf.initialize_all_variables()
        sess = tf.Session()
        sess.run(init)
        tf.logging.set_verbosity(tf.logging.ERROR)
        
        tokens = sent.tokens
        tensor_embedding = elmo(inputs={"tokens": [tokens], "sequence_len": [len(tokens)]}, signature="tokens", as_dict=True)[
        "elmo"]
        embedding = sess.run(tensor_embedding[0])
        
        file_name = str(convert_to_number(sent.raw))
        print(file_name)
        
        with open(os.path.join('ml_models', 'elmo', 'sick', '1', 'sentence.txt' ), "a") as sent_file:
            sent_file.write(sent.raw + "\n")
        
        np.save( os.path.join('ml_models', 'elmo', 'sick', '1', file_name), embedding) 
        
        
        update_progress_bar(counter, max_length, prefix='Progress:', suffix='Complete', length=100)
        counter = counter + 1

In [15]:
from preprocessing.load_data import download_and_load_sts_data, download_and_load_sick_dataset

sick_all, sick_train, sick_test, sick_dev = download_and_load_sick_dataset()
sick_all[:10]

,idx,sent_1,sent_2,sim,label
0,1,A group of kids is playing in a yard and an ol...,A group of boys in a yard is playing and a man...,4.5,NEUTRAL
1,2,A group of children is playing in the house an...,A group of kids is playing in a yard and an ol...,3.2,NEUTRAL
2,3,The young boys are playing outdoors and the ma...,The kids are playing outdoors near a man with ...,4.7,ENTAILMENT
3,5,The kids are playing outdoors near a man with ...,A group of kids is playing in a yard and an ol...,3.4,NEUTRAL
4,9,The young boys are playing outdoors and the ma...,A group of kids is playing in a yard and an ol...,3.7,NEUTRAL
5,12,Two dogs are fighting,Two dogs are wrestling and hugging,4.0,NEUTRAL
6,14,A brown dog is attacking another animal in fro...,Two dogs are fighting,3.5,NEUTRAL
7,18,A brown dog is attacking another animal in fro...,Two dogs are wrestling and hugging,3.2,NEUTRAL
8,25,Nobody is riding the bicycle on one wheel,A person in a black jacket is doing tricks on ...,2.8,NEUTRAL
9,26,A person is riding the bicycle on one wheel,A man in a black jacket is doing tricks on a m...,3.7,NEUTRAL


In [16]:
from models.sentence import Sentence

sentences1 = [Sentence(s) for s in sick_train['sent_1']]

save_vectors(sentences1[:10])

458212660998589423838483065212366987539804588811194837274233044429723162629710689506391152405390922527923525781343317678697251388770331674337346720608814219863761092619418981389664545468353814593
2163848112326249791906875313132085195676370823532807856858075316129154721337321084672016418698164900089539067454609333649022552927563415814539415585042446887168779482199475446202001228900480698541174232042648596062273
1627500836740216184407129153839322599401199327100290606839800656581632217962057764938380697744272346389500358060018379723031857822262530977722718160251349076
17163929413917009011261950530571469209365340449799440385078881795917168832510546030283918507652007687570166460381298749058934868
1627500836740216184407129153839322599401199327100290606839800656581632217962057764938380697744272346389500358060018379723031857822262530977722718160251349076
151165009348671460628149725582257340610250976229204------------------------------------------------------------| 40.0% Complete
2597106170458

In [33]:
d1 = np.load(os.path.join('ml_models', 'elmo', '1','1.npy'))
d2 = np.load(os.path.join('ml_models', 'elmo', '1','2.npy'))


In [34]:
sim = cosine_similarity(d1, d2)[0][0]
print(sim)

0.7171137


In [ ]:
from utility.hashing import convert_to_number, convert_from_number


def get_similarity(sent1, sent2, doc_freqs=None, use_stoplist=False):

    if doc_freqs is not None:
        N = doc_freqs["NUM_DOCS"]

    tokens1 = sent1.tokens_without_stop if use_stoplist else sent1.tokens
    tokens2 = sent2.tokens_without_stop if use_stoplist else sent2.tokens

    tokfreqs1 = Counter(tokens1)
    tokfreqs2 = Counter(tokens2)

    weights1 = [tokfreqs1[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                for token in tokens1] if doc_freqs else None

    weights2 = [tokfreqs2[token] * math.log(N / (doc_freqs.get(token, 0) + 1))
                for token in tokens2] if doc_freqs else None

    number1 = convert_to_number(sent1)
    number2 = convert_to_number(sent2)
                                 
    embeddings1 = 

    embeddings2 = 
    
    embedding1 = np.average([embedding for embedding in sess.run(embeddings1[0])], axis=0,
                            weights=weights1).reshape(1, -1)

    embedding2 = np.average([embedding for embedding in sess.run(embeddings2[0])], axis=0,
                            weights=weights2).reshape(1, -1)

    sim = cosine_similarity(embedding1, embedding2)[0][0]

    sess.close()

    return sim